In [1]:
import cudf as pd
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas

In [2]:
adv_data = pd.read_csv('data/adv_data.csv')
event_data = pd.read_csv('data/event_data.csv')
users_data = pd.read_csv('data/users_data.csv')

In [3]:
adv_data.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 9 columns):
 #   Column              Dtype
---  ------              -----
 0   user_id             object
 1   campaign_type       object
 2   campaign_start_dtm  object
 3   campaign_end_dtm    object
 4   source_medium       object
 5   utm_campaign        object
 6   interface           object
 7   currency            object
 8   campaign_cost       float64
dtypes: float64(1), object(8)
memory usage: 1.6+ GB


In [4]:
usdrub = 90.41

In [5]:
# event_data.loc[event_data['currency'] == 'USD']

In [6]:
def convert_to_rub(row):
    x = row['payout']
    if row['currency'] == 'USD':       
        return x * usdrub
    else:
        return x

In [7]:
def convert_campaign_cost_to_rub(row):
    x = row['campaign_cost']
    if row['currency'] == 'USD':       
        return x * usdrub
    else:
        return x

In [8]:
event_data['payout'] = event_data.apply(convert_to_rub, axis=1)
adv_data['campaign_cost'] = adv_data.apply(convert_campaign_cost_to_rub, axis=1)

In [9]:
event_data.drop('currency', inplace=True, axis=1)
adv_data.drop('currency', inplace=True, axis=1)

In [10]:
# adv_data['campaign_type'].value_counts()

In [11]:
# event_data['event_type'].value_counts()

In [12]:
# event_data['event_status'].unique()

In [13]:
users_data = users_data[~users_data['user_id'].isna()]

In [14]:
# event_data.drop('id', inplace=True, axis=1)

In [15]:
event_data['payout'].fillna(0.0, inplace=True)

In [16]:
adv_data['campaign_cost'].fillna(0.0, inplace=True)

In [17]:
users_data.drop('delete_dtm', axis=1, inplace=True)

In [18]:
# event_data_dup_id = event_data.loc[event_data.duplicated(subset='id')]['id'].to_arrow()

In [19]:
# event_data_duplicates = event_data[event_data['id'].isin(event_data_dup_id)]

In [20]:
# event_data_duplicates_phones = event_data_duplicates['user_phone'].to_arrow()

In [21]:
# users_data[users_data['user_phone'].isin(event_data_duplicates_phones)]

In [22]:
# len(users_data['user_id'].unique())

In [23]:
# len(users_data)

In [24]:
# len(users_data) - len(users_data['user_id'].unique())

In [25]:
# len(event_data) - len(event_data['id'].unique())

In [26]:
# users_data['user_id'].isin(users_data['user_id'].unique().to_arrow())

In [27]:
event_phones = event_data['user_phone'].to_arrow()
np_test = np.zeros_like(event_phones)

user_phones = users_data['user_phone'].to_arrow()
user_test = np.zeros_like(user_phones)

In [28]:
for i, val in enumerate(event_phones):
    np_test[i] = str(val).strip('+')
    
for i, val in enumerate(user_phones):
    user_test[i] = str(val).strip('+')

In [29]:
event_data['user_phone'] = np_test
users_data['user_phone'] = user_test

In [30]:
del np_test, user_test

In [31]:
users_uuids = users_data['user_id'].unique().to_arrow()

In [32]:
adv_data = adv_data[adv_data['user_id'].isin(users_uuids)]

In [33]:
adv_data.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 9999908 entries, 0 to 9999999
Data columns (total 8 columns):
 #   Column              Dtype
---  ------              -----
 0   user_id             object
 1   campaign_type       object
 2   campaign_start_dtm  object
 3   campaign_end_dtm    object
 4   source_medium       object
 5   utm_campaign        object
 6   interface           object
 7   campaign_cost       float64
dtypes: float64(1), object(7)
memory usage: 1.6+ GB


In [34]:
user_phones = users_data['user_phone'].unique().to_arrow()

In [35]:
users_data.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 7384334 entries, 0 to 7384333
Data columns (total 3 columns):
 #   Column            Dtype
---  ------            -----
 0   user_phone        object
 1   user_id           object
 2   registration_dtm  object
dtypes: object(3)
memory usage: 626.8+ MB


In [36]:
event_data.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 8 columns):
 #   Column        Dtype
---  ------        -----
 0   id            object
 1   user_phone    object
 2   event_type    object
 3   event_dtm     object
 4   event_status  object
 5   update_dtm    object
 6   payout        float64
 7   product_type  object
dtypes: float64(1), object(7)
memory usage: 1.4+ GB


In [37]:
event_data = event_data[event_data['user_phone'].isin(user_phones)]

In [38]:
event_data.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 9998930 entries, 0 to 9999999
Data columns (total 8 columns):
 #   Column        Dtype
---  ------        -----
 0   id            object
 1   user_phone    object
 2   event_type    object
 3   event_dtm     object
 4   event_status  object
 5   update_dtm    object
 6   payout        float64
 7   product_type  object
dtypes: float64(1), object(7)
memory usage: 1.5+ GB


In [39]:
adv_data_plus_user_data = adv_data.merge(users_data, how='left', on='user_id')

In [40]:
adv_data_plus_user_data.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 9999908 entries, 0 to 9999907
Data columns (total 10 columns):
 #   Column              Dtype
---  ------              -----
 0   user_id             object
 1   campaign_type       object
 2   campaign_start_dtm  object
 3   campaign_end_dtm    object
 4   source_medium       object
 5   utm_campaign        object
 6   interface           object
 7   campaign_cost       float64
 8   user_phone          object
 9   registration_dtm    object
dtypes: float64(1), object(9)
memory usage: 1.9+ GB


In [41]:
# adv_data - campaign_type, event_data - event_type

In [42]:
all_data = adv_data_plus_user_data.merge(event_data, how='inner', right_on=['user_phone', 'event_type'], left_on=['user_phone', 'campaign_type'])

In [43]:
del users_uuids, adv_data, users_data, user_phones, event_data, adv_data_plus_user_data

In [44]:
def parse_dates(row):
    return pd.to_datetime(row['campaign_start_dtm'])

In [45]:
date_to_fill = '2050-01-01 00:00:00.000000'

In [46]:
all_data['campaign_start_dtm'] = pd.to_datetime(all_data['campaign_start_dtm'])
all_data['registration_dtm'] = pd.to_datetime(all_data['registration_dtm'])
all_data['event_dtm'] = pd.to_datetime(all_data['event_dtm'])
all_data['update_dtm'].fillna(date_to_fill, inplace=True)
all_data['update_dtm'] = pd.to_datetime(all_data['update_dtm'])

In [47]:
all_data.drop('campaign_end_dtm', axis=1, inplace=True)

In [48]:
all_data['time_from_reg_till_event'] = all_data['event_dtm'] - all_data['registration_dtm']

In [49]:
all_data['time_from_camp_start_till_event'] = all_data['event_dtm'] - all_data['campaign_start_dtm']

In [50]:
all_data['campaign_cost'].fillna(0.0, inplace=True)

In [51]:
all_data.drop(['campaign_start_dtm', 'registration_dtm', 'event_dtm', 'update_dtm'], axis=1, inplace=True)

In [52]:
all_data.set_index(['user_id', 'campaign_type'])

source_medium  \
user_id                          campaign_type                     
1cf2827f8084e0c55e94cb675551bb4d show           yandex / organic   
                                 click          google / organic   
b1dda5296d4c760494c952551d9ed2f6 show             not-set / none   
70391e123115e9d0f463495adfa9ff91 show           yandex / organic   
                                 click          yandex / organic   
...                                                          ...   
5dd92af0b4583f0c398213de68e6f791 show               yandex / cpc   
01ce95aa0c2a6880c78bb5a90303549c show              regular / sms   
c28151d62405e210012dee3a153bcd21 show           google / organic   
bc23a8b4796dfc2a50d903b41e051312 show              trigger / sms   
d50ea2a785d4e801976a688af5038574 show           yandex / organic   

                                                             utm_campaign  \
user_id                          campaign_type                              
1cf2827f8084e0c55e94cb675551bb4d show                    client_case_mind   
                                 click           client_target_clickarrow   
b1dda5296d4c760494c952551d9ed2f6 show                                <NA>   
70391e123115e9d0f463495adfa9ff91 show            client_rebrand_megamaket   
                                 click                      bk_adv_client   
...                                                                   ...   
5dd92af0b4583f0c398213de68e6f791 show               light_retarget_market   
01ce95aa0c2a6880c78bb5a90303549c show              mind_rebrand_megamaket   
c28151d62405e210012dee3a153bcd21 show           operator_response_rebrand   
bc23a8b4796dfc2a50d903b41e051312 show            clickarrow_response_case   
d50ea2a785d4e801976a688af5038574 show               case_response_rebrand   

                                               interface  campaign_cost  \
user_id                          campaign_type                            
1cf2827f8084e0c55e94cb675551bb4d show                app           0.00   
                                 click               app         182.43   
b1dda5296d4c760494c952551d9ed2f6 show                web           0.00   
70391e123115e9d0f463495adfa9ff91 show                app           0.00   
                                 click               web           0.00   
...                                                  ...            ...   
5dd92af0b4583f0c398213de68e6f791 show                web           0.00   
01ce95aa0c2a6880c78bb5a90303549c show                app           0.00   
c28151d62405e210012dee3a153bcd21 show                app           0.00   
bc23a8b4796dfc2a50d903b41e051312 show                app           0.00   
d50ea2a785d4e801976a688af5038574 show                web           0.00   

                                                 user_phone  \
user_id                          campaign_type                
1cf2827f8084e0c55e94cb675551bb4d show           79110685723   
                                 click          79110685723   
b1dda5296d4c760494c952551d9ed2f6 show           79169256811   
70391e123115e9d0f463495adfa9ff91 show           79143423444   
                                 click          79143423444   
...                                                     ...   
5dd92af0b4583f0c398213de68e6f791 show           79136226770   
01ce95aa0c2a6880c78bb5a90303549c show           79100032208   
c28151d62405e210012dee3a153bcd21 show           79175795131   
bc23a8b4796dfc2a50d903b41e051312 show           79173299451   
d50ea2a785d4e801976a688af5038574 show           79183086727   

                                                                                               id  \
user_id                          campaign_type                                                      
1cf2827f8084e0c55e94cb675551bb4d show           9584bfbbb19f59401f60d909a5e5e6101399c631f6e6f3...   
                                 click          30b05e38d28554dad

In [53]:
all_data.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 8482145 entries, 0 to 8482144
Data columns (total 14 columns):
 #   Column                           Dtype
---  ------                           -----
 0   user_id                          object
 1   campaign_type                    object
 2   source_medium                    object
 3   utm_campaign                     object
 4   interface                        object
 5   campaign_cost                    float64
 6   user_phone                       object
 7   id                               object
 8   event_type                       object
 9   event_status                     object
 10  payout                           float64
 11  product_type                     object
 12  time_from_reg_till_event         timedelta64[ns]
 13  time_from_camp_start_till_event  timedelta64[ns]
dtypes: float64(2), object(10), timedelta64[ns](2)
memory usage: 1.8+ GB


In [54]:
# product_types = all_data['product_type'].unique().to_arrow()

In [55]:
# product_types

In [62]:
# loan_data = all_data.loc[all_data['product_type'] == 'loan']
# deposit_data = all_data.loc[all_data['product_type'] == 'deposits']
# mortgage_data = all_data.loc[all_data['product_type'] == 'mortgage']

In [65]:
# del all_data

In [64]:
# del mortgage_data

In [63]:
# mortgage_data.to_csv('data/mortgage_data.csv')


In [57]:
# loan_data.to_csv('data/loan_data.csv')


In [60]:
# deposit_data.to_csv('data/deposit_data.csv')